### Desafio
####  Moises Quiroz Diaz

### Ejercicio 1: Preliminares y datos perdidos

    Genere una instancia en AWS EMR y habilite un puerto dinámico para utilizar un notebook
    desde JupyterHub.
    Genere los objetos SparkConf y SparkContext .
    Utilizando Spark , importe el archivo household_power_consumption.txt a un objeto RDD.
    El primer registro del archivo corresponde a las etiquetas asociadas a cada columna. Elimínelo
    de su RDD.
    Cada campo dentro del registro está separado por un ; . Separe cada campo del registro por
    éste caracter.
    Reporte las siguientes métricas:

In [1]:
sc #Se asume se autogeneran el conf y contex al estar trabajando dentro de aws con una instancia de spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1575460809126_0001,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
df = sc.textFile('s3://bigdata-desafio/challenges/u4lec1/household_power_consumption.txt')

In [3]:
header = df.first()
header = sc.parallelize([header])
df = df.subtract(header)

In [4]:
df_proc = df\
            .map(lambda x: x.split(';'))\
            .map(lambda x: (str(x[0]),str(x[1]),str(x[2]),str(x[3]),str(x[4]),str(x[5]),str(x[6]),str(x[7]),str(x[8])))


    La cantidad total de registros antes de eliminar los datos perdidos.
    La cantidad total de registros nulos.
    La cantidad de registros nulos por año. ¿En qué año hubo más registros nulos?
    La cantidad de registros nulos por mes para el año con una mayor cantidad de registros
    nulos. ¿En qué mes hubo más registros nulos?

In [5]:
print('la cantidad total de registros antes de eliminar es:',df_proc.count())

la cantidad total de registros antes de eliminar es: 2075259

In [6]:
nulos = df_proc.keyBy(lambda x: (x[0],x[1]))\
    .filter(lambda x: '?' in x[1])
print('la cantidad total de registros con datos nulos es:',nulos.count())

la cantidad total de registros con datos nulos es: 25979

In [7]:
perdidos_año = nulos\
                    .map(lambda x:x[0][0].split('/'))\
                    .map(lambda x: x[2])\
                    .map(lambda x: (x,1))\
                    .reduceByKey(lambda x,y : x+y)\
                    .sortBy(lambda x: x[1],ascending=False)

perdidos_año_num = perdidos_año.collect()

In [8]:
print('El año con mas perdidos fue el 2010','\n','*'*50)
for anio,cantidad in perdidos_año_num:
    print('Para el año',anio,'los datos perdidos fueron',cantidad)

El año con mas perdidos fue el 2010 
 **************************************************
Para el año 2010 los datos perdidos fueron 17629
Para el año 2009 los datos perdidos fueron 4280
Para el año 2007 los datos perdidos fueron 3931
Para el año 2008 los datos perdidos fueron 135
Para el año 2006 los datos perdidos fueron 4

In [9]:
get_nulos_2010 = nulos\
                    .filter(lambda x:x[0][0].split('/')[2] == '2010')\
                    .map(lambda x:x[0][0].split('/'))\
                    .map(lambda x: x[1])\
                    .map(lambda x: (x,1))\
                    .reduceByKey(lambda x,y : x+y)\
                    .sortBy(lambda x: x[1],ascending=False)

get_nulos_2010_num = get_nulos_2010.collect()

In [10]:
print('El año con mas perdidos fue el mes 8','\n','*'*50)
for mes,cantidad in get_nulos_2010_num:
    print('Para el mes',mes,'los datos perdidos fueron',cantidad)

El año con mas perdidos fue el mes 8 
 **************************************************
Para el mes 8 los datos perdidos fueron 7226
Para el mes 9 los datos perdidos fueron 5237
Para el mes 1 los datos perdidos fueron 3131
Para el mes 3 los datos perdidos fueron 2027
Para el mes 2 los datos perdidos fueron 2
Para el mes 6 los datos perdidos fueron 2
Para el mes 4 los datos perdidos fueron 1
Para el mes 7 los datos perdidos fueron 1
Para el mes 5 los datos perdidos fueron 1
Para el mes 10 los datos perdidos fueron 1

### Ejercicio 2: Preparación de los datos

Genere un objeto donde se encuentren sólo aquellos datos sin registros perdidos.

In [11]:
df_2 = df_proc.keyBy(lambda x: (x[0],x[1]))\
    .filter(lambda x: '?' not in x[1]).map(lambda x:x[1])

Con el objeto sin datos perdidos, genere un objeto RDD donde los registro tengan la siguiente
estructura:

In [12]:
df_2.take(1)

[('16/12/2006', '17:34:00', '4.448', '0.498', '232.860', '19.600', '0.000', '1.000', '17.000')]

In [97]:
df_proc_2 = df_2\
            .map(lambda x: (x[0].split('/'),x[2],x[3],x[4],x[5],x[6],x[7],x[8]))\
            .map(lambda x: (x[0][2],x[0][1],x[0][0],x[1],x[2],x[3],x[4],x[5],x[6],x[7]))\
            .map(lambda x: (int(x[0]),int(x[1]),int(x[2]),float(x[3]),float(x[4]),float(x[5]),float(x[6]),float(x[7]),float(x[8]),float(x[9])))\
            .sortBy(lambda x: x[0],ascending=True)

print(df_proc_2.take(1))




[(2006, 12, 16, 3.666, 0.528, 235.68, 15.8, 0.0, 1.0, 17.0)]

Devuelva con collect las primeras 5 observaciones

In [14]:
for i in df_proc_2.collect()[0:5]:
    print(i)

(2006, 12, 16, 3.668, 0.51, 233.99, 15.8, 0.0, 1.0)
(2006, 12, 16, 3.758, 0.076, 234.17, 16.4, 0.0, 0.0)
(2006, 12, 16, 4.512, 0.0, 233.62, 19.2, 0.0, 0.0)
(2006, 12, 16, 4.058, 0.2, 234.68, 17.6, 0.0, 0.0)
(2006, 12, 16, 6.474, 0.144, 231.85, 27.8, 0.0, 37.0)

### Ejercicio 3: Patrones globales

    Extraiga la media, desviación estándar e intervalos de confianza (a .5 desviaciones estándar)
    para las columnas global_activity_power , global_reactive_power , voltage y
    global_intensity .

##### global_activity_power

In [35]:
global_mean = df_proc_2\
                    .map(lambda x: x[3])\
                    .reduce(lambda x,y: x+y)/df_proc_2.count()
print('la media de global_activity_power es:',global_mean)

la media de global_activity_power es: 1.091615036500582

In [41]:
global_sd =  df_proc_2\
                .map(lambda x:x[3])\
                .map(lambda x: x - global_mean)\
                .map(lambda x: x**2)\
                .reduce(lambda x,y: x+y)/(df_proc_2.count()-1)
print('La desviacion estandar correspode a:',global_sd**(1/2))

La desviacion estandar correspode a: 1.0572941610939288

In [40]:
print('El intervalo de confianza corresponde a (',global_mean-.5*global_sd,global_mean-.5*global_sd,')')

El intervalo de confianza corresponde a ( 0.5326795649589189 0.5326795649589189 )

In [78]:
def estadisticos(indice):
    temp_df = df_proc_2\
                    .map(lambda x: x[indice])
    mean = temp_df.mean()
    sd = temp_df.stdev()
    low = mean - .5*sd
    up = mean + .5*sd
    print('la media corresponde a:',mean)
    print('la desviacion estandar corresponde a:',sd)
    print('El intervalo de confianza corresponde a (',low,up,')')

###### global_reactive_power

In [43]:
estadisticos(4)

la media corresponde a: 0.12371447630387347
la desviacion estandar corresponde a: 0.11272195204788803
El intervalo de confianza corresponde a ( 0.06735350027992945 0.1800754523278175 )

###### voltage

In [44]:
estadisticos(5)

la media corresponde a: 240.8398579745102
la desviacion estandar corresponde a: 3.2399858884907893
El intervalo de confianza corresponde a ( 239.21986503026483 242.4598509187556 )

###### global_intensity 

In [45]:
estadisticos(6)

la media corresponde a: 4.6277593105870904
la desviacion estandar corresponde a: 4.444395175406088
El intervalo de confianza corresponde a ( 2.4055617228840465 6.849956898290134 )

### Ejercicio 4: Patrón temporal

    Reporte el promedio de global_activity_power a nivel mensual y anual. Genere un objeto
    con la acción collectAsMap() del RDD procesado.

In [76]:
total = df_proc_2.count()
prom_mes = df_proc_2.map(lambda x: (x[1],x[3]))\
                .reduceByKey(lambda x,y: x+y)\
                .map(lambda x: (x[0],x[1]/total)).collectAsMap()

for mes,prom in prom_mes.items():
    print('para el mes:',mes,'se tiene un promedio de:',prom)

para el mes: 6 se tiene un promedio de: 0.07517479504996832
para el mes: 12 se tiene un promedio de: 0.11328679829012811
para el mes: 1 se tiene un promedio de: 0.1251712357510941
para el mes: 7 se tiene un promedio de: 0.06097724273891266
para el mes: 2 se tiene un promedio de: 0.1032136916380406
para el mes: 8 se tiene un promedio de: 0.047635324601811184
para el mes: 3 se tiene un promedio de: 0.10607026370237621
para el mes: 9 se tiene un promedio de: 0.07977357218145009
para el mes: 4 se tiene un promedio de: 0.08639380953310453
para el mes: 10 se tiene un promedio de: 0.09905649984384787
para el mes: 5 se tiene un promedio de: 0.08970612215021889
para el mes: 11 se tiene un promedio de: 0.10515568101967883

In [77]:
prom_anual = df_proc_2.map(lambda x: (x[0],x[3]))\
                .reduceByKey(lambda x,y: x+y)\
                .map(lambda x: (x[0],x[1]/total)).collectAsMap()

for mes,prom in prom_anual.items():
    print('para el año:',mes,'se tiene un promedio de:',prom)

para el año: 2010 se tiene un promedio de: 0.23681249121642517
para el año: 2006 se tiene un promedio de: 0.02040388917083071
para el año: 2007 se tiene un promedio de: 0.28434776409275947
para el año: 2008 se tiene un promedio de: 0.2756544220408781
para el año: 2009 se tiene un promedio de: 0.27439646997969647


    
    Reporte todas aquellos registros que presenten un promedio de global_activity_power
    superior a .5 desviaciones estándares. ¿A qué meses y años corresponden?
    
    Reporte todos aquellos registros que presenten un promedio de global_activity_power
    inferior a .5 desviaciones estándares. ¿A qué meses y años corresponden?

In [79]:
# Estos datos se obtienen en la pregunta 3
low = 0.5326795649589189
up = 0.5326795649589189

In [80]:
superior = df_proc_2.filter(lambda x: x[3]>up)\
                .map(lambda x: (x[0],x[1],x[3]))
inferior = df_proc_2.filter(lambda x: x[3]<low)\
                .map(lambda x: (x[0],x[1],x[3]))

##### Superior

In [88]:
#mes
for mes,val in superior.map(lambda x: (x[1],x[2])).collectAsMap().items():
    print('el mes corresponde a:',mes,'con un valor de:',val )

el mes corresponde a: 12 con un valor de: 1.792
el mes corresponde a: 1 con un valor de: 0.932
el mes corresponde a: 2 con un valor de: 0.826
el mes corresponde a: 3 con un valor de: 0.562
el mes corresponde a: 4 con un valor de: 0.9
el mes corresponde a: 5 con un valor de: 0.62
el mes corresponde a: 6 con un valor de: 3.73
el mes corresponde a: 7 con un valor de: 0.666
el mes corresponde a: 8 con un valor de: 1.228
el mes corresponde a: 9 con un valor de: 1.17
el mes corresponde a: 10 con un valor de: 2.46
el mes corresponde a: 11 con un valor de: 1.024

In [89]:
#año
for anio,val in superior.map(lambda x: (x[0],x[2])).collectAsMap().items():
    print('el anio corresponde a:',anio,'con un valor de:',val )

el anio corresponde a: 2006 con un valor de: 2.586
el anio corresponde a: 2007 con un valor de: 1.732
el anio corresponde a: 2008 con un valor de: 0.556
el anio corresponde a: 2009 con un valor de: 1.792
el anio corresponde a: 2010 con un valor de: 1.024

##### Inferior

In [92]:
for mes,val in inferior.map(lambda x: (x[1],x[2])).collectAsMap().items():
    print('el mes corresponde a:',mes,'con un valor de:',val )
print('\n','*'*100,'\n')
for anio,val in inferior.map(lambda x: (x[0],x[2])).collectAsMap().items():
    print('el anio corresponde a:',anio,'con un valor de:',val )

el mes corresponde a: 12 con un valor de: 0.49
el mes corresponde a: 1 con un valor de: 0.306
el mes corresponde a: 2 con un valor de: 0.392
el mes corresponde a: 3 con un valor de: 0.442
el mes corresponde a: 4 con un valor de: 0.348
el mes corresponde a: 5 con un valor de: 0.326
el mes corresponde a: 6 con un valor de: 0.422
el mes corresponde a: 7 con un valor de: 0.152
el mes corresponde a: 8 con un valor de: 0.434
el mes corresponde a: 9 con un valor de: 0.324
el mes corresponde a: 10 con un valor de: 0.444
el mes corresponde a: 11 con un valor de: 0.522

 **************************************************************************************************** 

el anio corresponde a: 2006 con un valor de: 0.428
el anio corresponde a: 2007 con un valor de: 0.442
el anio corresponde a: 2008 con un valor de: 0.48
el anio corresponde a: 2009 con un valor de: 0.356
el anio corresponde a: 2010 con un valor de: 0.514

### Ejercicio 5: Consumo energético general

In [103]:
ejer_5 = df_proc_2.map(lambda x: (x[0],x[1],x[2],x[3]*(1000/60)-x[7]-x[8]-x[9]))

In [ ]:
total = ejer_5.count()

In [104]:
prom_anual = ejer_5.map(lambda x: (x[0],x[3]))\
                .reduceByKey(lambda x,y: x+y)\
                .map(lambda x: (x[0],x[1]/total)).collectAsMap()

for anio,prom in prom_anual.items():
    print('para el año:',anio,'se tiene un promedio de:',prom)

para el año: 2010 se tiene un promedio de: 1.8637197942693506
para el año: 2006 se tiene un promedio de: 0.22337847113782994
para el año: 2007 se tiene un promedio de: 2.53359965451286
para el año: 2008 se tiene un promedio de: 2.43451792824787
para el año: 2009 se tiene un promedio de: 2.2594774587496715

In [107]:
prom_mes = ejer_5.map(lambda x: (x[1],x[3]))\
                .reduceByKey(lambda x,y: x+y)\
                .map(lambda x: (x[0],x[1]/total)).collectAsMap()

for mes,prom in prom_mes.items():
    print('para el mes:',mes,'se tiene un promedio de:',prom)

para el mes: 6 se tiene un promedio de: 0.5296201918072017
para el mes: 12 se tiene un promedio de: 1.0850610132989773
para el mes: 1 se tiene un promedio de: 1.1660060281334588
para el mes: 7 se tiene un promedio de: 0.49601440505934175
para el mes: 2 se tiene un promedio de: 0.9181230643608138
para el mes: 8 se tiene un promedio de: 0.3639266636737451
para el mes: 3 se tiene un promedio de: 0.9187439002966964
para el mes: 9 se tiene un promedio de: 0.6184473408546721
para el mes: 4 se tiene un promedio de: 0.7033761939152602
para el mes: 10 se tiene un promedio de: 0.8553110035394591
para el mes: 5 se tiene un promedio de: 0.6900617289975031
para el mes: 11 se tiene un promedio de: 0.9700017729804252

In [108]:
prom_dia = ejer_5.map(lambda x: (x[2],x[3]))\
                .reduceByKey(lambda x,y: x+y)\
                .map(lambda x: (x[0],x[1]/total)).collectAsMap()

for dia,prom in prom_dia.items():
    print('para el dia:',dia,'se tiene un promedio de:',prom)

para el dia: 18 se tiene un promedio de: 0.3135484820685004
para el dia: 24 se tiene un promedio de: 0.3210408045752651
para el dia: 30 se tiene un promedio de: 0.2741726525088479
para el dia: 6 se tiene un promedio de: 0.3132560216271068
para el dia: 12 se tiene un promedio de: 0.2966495386997707
para el dia: 7 se tiene un promedio de: 0.3035895046064953
para el dia: 13 se tiene un promedio de: 0.28871065284717834
para el dia: 19 se tiene un promedio de: 0.29700463902769064
para el dia: 25 se tiene un promedio de: 0.3069840626951916
para el dia: 31 se tiene un promedio de: 0.19768138077763814
para el dia: 1 se tiene un promedio de: 0.3004688150635016
para el dia: 20 se tiene un promedio de: 0.30882903588902916
para el dia: 26 se tiene un promedio de: 0.2818557737351675
para el dia: 2 se tiene un promedio de: 0.2862574497709761
para el dia: 8 se tiene un promedio de: 0.30963566065479836
para el dia: 14 se tiene un promedio de: 0.29817653029356606
para el dia: 3 se tiene un promedio de: